# Бартов Олег Борисович
## Решение задачи по предсказанию набора специализаций для вакансии
### 2020-07-27

In [21]:
#стандартные библиотеки
import pandas as pd
import numpy as np

#для загрузки данных
import json
import gzip

#для очистки htmp-тегов
from bs4 import BeautifulSoup
#регулярные выражения
import re
#стемминг
from nltk.stem.snowball import SnowballStemmer

#прогресс выполнения
from tqdm.notebook import tqdm

#для векторайзеры
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#пакетная кластеризация KMeans
from sklearn.cluster import MiniBatchKMeans

#для мультилейбл
from sklearn.preprocessing import MultiLabelBinarizer

#бустинг
import catboost
#сбор мусора
import gc

#процедура для загрузки вакансий в словарь
def read_vacancies_part(part):
    with gzip.open(f'vacancies-{part:02}.json.gz', 'r') as fp:
        return json.loads(fp.read())

# №1 Предобработка данных

In [4]:
%%time

#объявляем стемминг
stemmer = SnowballStemmer('russian')
#запускаем цикл по частям
for part_num in range(1, 11):
    
    #грузим часть
    part = read_vacancies_part(part_num)
    print('Часть '+str(part_num))
    print(f'Всего вакансий в части: {len(part):,d}')
          
    #запускаем цикл для препроцессинга текста
    for k in tqdm(part):
          
        #работаем с наименованием вакансии
        transf_name = part[k]['name']
        #приводим все к нижнему регистру
        transf_name = transf_name.lower()
        #убираем знаки препинания и цифры
        transf_name = re.sub(r"[^a-zа-я]+",' ', transf_name)
        #убираем лишние пробелы
        transf_name = transf_name.strip()
        #делаем стемминг - получается массив
        transf_name = [stemmer.stem(word) for word in transf_name.split(' ')]
        #возвращаем строку из массива обратно
        transf_name = ' '.join(map(str, transf_name))
        #переписываем обработанное наименование вакансии
        part[k]['name'] = transf_name

        
        #теперь работаем с описанием
        #убираем html-теги
        transf_desc = BeautifulSoup(part[k]['description'], "lxml").text
        #приводим все к нижнему регистру
        transf_desc = transf_desc.lower()
        #убираем знаки препинания и цифры
        transf_desc = re.sub(r"[^a-zа-я]+",' ', transf_desc)
        #убираем лишние пробелы
        transf_desc = transf_desc.strip()
        #делаем стемминг - получается массив
        transf_desc = [stemmer.stem(word) for word in transf_desc.split(' ')]        
        #возвращаем строку из массива обратно
        transf_desc = ' '.join(map(str, transf_desc))
        #переписываем обработанное описание вакансии
        part[k]['description'] = transf_desc
        
        
        #работаем с ключевыми навыками вакансии
        transf_skills = part[k]['key_skills']
        #из массива в строку
        transf_skills = ' '.join(map(str, transf_skills))
        #приводим все к нижнему регистру
        transf_skills = transf_skills.lower()
        #убираем лишние пробелы
        transf_skills = transf_skills.strip()
        #делаем стемминг - получается массив
        transf_skills = [stemmer.stem(word) for word in transf_skills.split(' ')]
        #возвращаем строку из массива обратно
        transf_skills = ' '.join(map(str, transf_skills))
        part[k]['key_skills'] = transf_skills
        
        #решаем вопрос с зарплатой - это должен стать один признак
        salary = part[k]['compensation_to']
        if salary is None:
            salary = part[k]['compensation_from']
        if salary is None:
            salary = 0
        part[k]['salary'] = salary
        #убираем лишнее
        part[k].pop('compensation_to')
        part[k].pop('compensation_from')
    
    #записываем часть, которую преобразовали в файл, дальше будем эти файлы объединять
    pd.DataFrame(part).T.to_csv('df_stem'+str(part_num)+'.csv',sep=';')

Часть 1
Всего вакансий в части: 300,000



Часть 2
Всего вакансий в части: 300,000



Часть 3
Всего вакансий в части: 300,000



Часть 4
Всего вакансий в части: 300,000



Часть 5
Всего вакансий в части: 300,000


...


Часть 7
Всего вакансий в части: 300,000



Часть 8
Всего вакансий в части: 300,000



Часть 9
Всего вакансий в части: 300,000



Часть 10
Всего вакансий в части: 212,650



Wall time: 4h 35min 4s


### объединяем обработанные части в единый датасет

In [5]:
%%time
df = pd.read_csv('df_stem1.csv',sep=';') #df_part1 #df_clean_morhp1
print(1)
df = df.append(pd.read_csv('df_stem2.csv',sep=';')) #df_part2 #df_clean_morhp2
print(2)
df = df.append(pd.read_csv('df_stem3.csv',sep=';')) #df_part3 #df_clean_morhp3
print(3)
df = df.append(pd.read_csv('df_stem4.csv',sep=';')) #df_part4 #df_clean_morhp4
print(4)
df = df.append(pd.read_csv('df_stem5.csv',sep=';')) #df_part5 #df_clean_morhp5
print(5)
df = df.append(pd.read_csv('df_stem6.csv',sep=';')) #df_part6 #df_clean_morhp6
print(6)
df = df.append(pd.read_csv('df_stem7.csv',sep=';')) #df_part7 #df_clean_morhp7
print(7)
df = df.append(pd.read_csv('df_stem8.csv',sep=';')) #df_part8 #df_clean_morhp8
print(8)
df = df.append(pd.read_csv('df_stem9.csv',sep=';')) #df_part9 #df_clean_morhp9
print(9)
df = df.append(pd.read_csv('df_stem10.csv',sep=';')) #df_part10 #df_clean_morhp10
print(10)

df.rename(columns={'Unnamed: 0': 'vac_id'}, inplace=True) #важная колонка - ключ
df.reset_index(inplace=True) #так как объединение, имеет смысл сбросить индекс
del df['index'] #лишний столбец

#сразу формируем категориальные признаки

#зп
df['sal_clust'] = (df['salary']/1000).astype('int64').astype('category').cat.codes
#валюта
df['currency'].fillna('RUR',inplace=True) #если не указано, пусть будет рубль
df['currency'] = df['currency'].astype('category').cat.codes
#остальные просто преобразуем в категории
df['employer'] = df['employer'].astype('category').cat.codes
df['employment'] = df['employment'].astype('category').cat.codes
df['work_schedule'] = df['work_schedule'].astype('category').cat.codes
df['work_experience'] = df['work_experience'].astype('category').cat.codes

#записываем объединенный датасет файл
df.to_csv('df_stem.csv',sep=';',index=False)
df.head()

1
2
3
4
5
6
7
8
9
10
Wall time: 1min 38s


,vac_id,name,description,area_id,creation_date,employment,work_schedule,work_experience,currency,key_skills,employer,salary,sal_clust
0,1,администратор торгов зал,обязан администратор торгов зал контролир техн...,26,2019-01-24,0,2,0,6,NaN,17253,24000,24
1,2,супервайзер команд служб поддержк пользовател,в связ с расширен в международн компан olx gro...,160,2019-07-26,0,2,0,6,NaN,250208,0,0
2,3,системн администратор,для обеспечен техническ поддержк объект европе...,1002,2019-04-15,3,2,0,6,NaN,24031,0,0
3,4,специалист по закупк,обязан организац и проведен закупок в рамк фз ...,22,2019-07-12,0,2,3,6,делов переписк заключен договор делов общен те...,304424,36000,36
4,5,уборщик помещен уборщиц,рестора o d i приглаша на работ уборщик помеще...,1002,2019-01-17,0,2,0,1,NaN,199788,600,0


# №2 Кластеризация

In [6]:
%%time
#читаем сохраненный ранее файл (лучше предобработку запускать отдельно из-за проблем с памятью, когда ее мало)
df = pd.read_csv('df_stem.csv',sep=';')

#выполняем преобразование, иначе в векторайзер не залезает
df['name'] = df['name'].astype('str')
df['description'] = df['description'].astype('str')
df['key_skills'] = df['key_skills'].astype('str')

#смотрим, что прочитали именно то, что нужно
df.head(10)

Wall time: 52.6 s


,vac_id,name,description,area_id,employment,work_schedule,work_experience,currency,key_skills,employer,salary,sal_clust,date_feat
0,1,администратор торгов зал,обязан администратор торгов зал контролир техн...,26,0,2,0,6,NaN,17253,24000,24,0
1,2,супервайзер команд служб поддержк пользовател,в связ с расширен в международн компан olx gro...,160,0,2,0,6,NaN,250208,0,0,6
2,3,системн администратор,для обеспечен техническ поддержк объект европе...,1002,3,2,0,6,NaN,24031,0,0,3
3,4,специалист по закупк,обязан организац и проведен закупок в рамк фз ...,22,0,2,3,6,делов переписк заключен договор делов общен те...,304424,36000,36,6
4,5,уборщик помещен уборщиц,рестора o d i приглаша на работ уборщик помеще...,1002,0,2,0,1,NaN,199788,600,0,0
5,6,ведущ инженер,обязан взаимодейств с рсо в част ответ на запр...,47,0,2,1,6,NaN,144862,40000,40,7
6,7,грузчик в отдел доставок,фабрик корпусн мебел примет на работ грузчик в...,4,0,2,3,6,грузчик доставк мебел,222138,22000,22,8
7,8,прораб фасадн работ,должностн обязан руководств бригад монтажник о...,1,0,2,1,6,NaN,283886,60000,60,5
8,9,автомеханик автослесар моторист,обязан ремонт и техническ обслуживан легков ав...,54,0,4,2,6,NaN,2120,70000,70,10
9,10,administrative assistant,responsibilities maintaining the condition of ...,152,0,2,0,6,английск язык project management ms access ms ...,188947,0,0,8


### № 2.1 Формируем кластеры по наименованию

In [ ]:
%%time
#tf-idf на мешок слов наименования
vectorizer = TfidfVectorizer(min_df=2)
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1800, max_iter=10, init_size=5400, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_name_1800',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на биграммы наименования
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1500, max_iter=10, init_size=4500, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_2gram_name_1500',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на триграммы наименования
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(3,3))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1800, max_iter=10, init_size=5400, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_3gram_name_1800',minikmeans.labels_)

In [ ]:
%%time
#count на мешок слов наименования
vectorizer = CountVectorizer(min_df=2)
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1500, max_iter=10, init_size=4500, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_count_name_1500',minikmeans.labels_)

In [ ]:
%%time
#count на биграммы наименования
vectorizer = CountVectorizer(min_df=2, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1500, max_iter=10, init_size=4500, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_count_2gram_name_1500',minikmeans.labels_)

### №2.2 Формируем кластеры по описанию

In [ ]:
%%time
#tf-idf на мешок слов описания
vectorizer = TfidfVectorizer(min_df=5, max_df=100000)
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_description_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на мешок слов другой частотности описания
vectorizer = TfidfVectorizer(min_df=5, max_df=1000)
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=5000, max_iter=10, init_size=15000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_description_s_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на биграммы описания
vectorizer = TfidfVectorizer(max_df=50000,min_df=100, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_2gramm_description_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на триграммы описания
vectorizer = TfidfVectorizer(max_df=50000,min_df=100, ngram_range=(3,3))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)
#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_3gramm_description_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на мешок слов и биграммы описания совместно
vectorizer = TfidfVectorizer(max_df=50000,min_df=100, ngram_range=(1,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2400, max_iter=10, init_size=9600, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_both_description_2400',minikmeans.labels_)

### №2.3 Формируем кластеры по ключевым навыкам

In [ ]:
%%time
#tf-idf на мешок слов ключевых навыков
vectorizer = TfidfVectorizer(min_df=2)
matrix = vectorizer.fit_transform(df['key_skills'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_skills_2000',minikmeans.labels_)

# №3 Обучение

In [49]:
%%time
#снова читаем сохраненный ранее файл (лучше обучение запускать отдельно из-за проблем с памятью, когда ее мало)
df = pd.read_csv('df_stem.csv',sep=';'
                 , usecols=['vac_id'
                           ,'employer','employment','area_id'
                           ,'work_experience','work_schedule','currency','sal_clust'
                           ,'creation_date']
                 , parse_dates=['creation_date'])

#дата подачи вакансии еще один категориальный признак, месяц подачи вакансии - пытаемся поймать сезонность
df['date_feat'] = df['creation_date'].dt.strftime('%Y-%m')
df['date_feat'] = df['date_feat'].astype('category').cat.codes
del df['creation_date']

#кластеры по наименованию
df['clust_name'] = np.load('clust_name_1800.npy')
df['clust_2gram_name'] = np.load('clust_2gram_name_1500.npy')
df['clust_3gram_name'] = np.load('clust_3gram_name_1800.npy')
df['clust_count_name'] = np.load('clust_count_name_1500.npy')
df['clust_count_2gram_name'] = np.load('clust_count_2gram_name_1500.npy')

#кластеры по описанию
df['clust_description'] = np.load('clust_description_2000.npy')
df['clust_description_s'] = np.load('clust_description_s_2000.npy')
df['clust_2gramm_description'] = np.load('clust_2gramm_description_2000.npy')
df['clust_3gramm_description'] = np.load('clust_3gramm_description_2000.npy')
df['clust_both_description'] = np.load('clust_both_description_2400.npy')

#кластер по ключевым навыкам
df['clust_skills'] = np.load('clust_skills_2000.npy')

#теперь грузим ответы
df_ts = pd.read_csv('train_labels.csv.gz', compression='gzip')
df_ts['spec'] = df_ts['specializations'].apply(lambda x: list(map(int, x[1:-1].split(','))))
del df_ts['specializations']
#добавляем ответы, формируя окончательный датасет
df = df.merge(df_ts,left_on='vac_id',right_on='vacancy_id',how='left')
del df['vacancy_id']

#список для обучения, весь список - категориальные переменные
cat_cols = ['employer','employment','area_id','work_experience','work_schedule','currency','sal_clust','date_feat'
           ,'clust_name','clust_2gram_name','clust_3gram_name','clust_count_name','clust_count_2gram_name' 
           ,'clust_description','clust_description_s','clust_2gramm_description','clust_3gramm_description','clust_both_description'
           ,'clust_skills'
           ]

#создаем набор для обучения
mlb = MultiLabelBinarizer()
df_train = df[~df['spec'].isna()]
mlb.fit(df_train['spec'])
y_train = mlb.transform(df_train['spec'])
X_train = df_train[cat_cols]

#набор для предсказания
df_test = df[df['spec'].isna()]
X_test = df_test[cat_cols]

Wall time: 37.9 s


### для обучения используем CatBoost

In [47]:
#создаем классификатор
cb = catboost.CatBoostClassifier(cat_features=cat_cols,iterations=300,eta=0.1,task_type='GPU'
                                 ,eval_metric='F1')

#посколько обучение идет долго, учим по 10 моделей, потом сохраняем их в файл, далее будем собирать
for n in range(10,621,10):
    y_result = []
    for i in tqdm(range(n-10,n)):
        
        #специализации, где совсем мало объектов не используем в обучении
        a = y_train[:,i]
        if len(a[a==1])<10:
            y_result.append(np.array([0]*1456325).astype('float16'))
            continue
        
        #собираем мусор
        gc.collect()
        
        #тренируем модель по текущей специализации
        print(i)
        cb.fit(X_train,y_train[:,i],verbose=50)
        
        #предсказываем массив с вероятностями
        y_result.append(np.round(cb.predict(X_test,prediction_type='Probability')[:,1],3).astype('float16'))      
        
        #при обучении смотрим, какие фичи работают для какой специализации
        ii=0
        imp = {}
        for nm in cb.feature_names_:
            imp[nm] = cb.feature_importances_[ii]
            ii+=1
        print(imp)
        print('')
    
    #сохраняем обученный десяток моделей
    np.save('y_result'+str(n),np.array(y_result))

42
0:	learn: 0.0872387	total: 101ms	remaining: 30.3s
50:	learn: 0.8418131	total: 4.62s	remaining: 22.5s
100:	learn: 0.8497347	total: 9.3s	remaining: 18.3s
150:	learn: 0.8515701	total: 14.1s	remaining: 13.9s
200:	learn: 0.8530386	total: 18.9s	remaining: 9.31s
250:	learn: 0.8540687	total: 23.7s	remaining: 4.63s
299:	learn: 0.8549167	total: 28.4s	remaining: 0us
{'employer': 5.381160175743924, 'employment': 0.10162060943521875, 'area_id': 0.34877578205792353, 'work_experience': 1.3880916152705023, 'work_schedule': 0.7931734936922054, 'currency': 0.29210283880079124, 'sal_clust': 0.26042547961931467, 'date_feat': 0.18070855966121124, 'clust_name': 9.406070635554087, 'clust_2gram_name': 0.697869925205182, 'clust_3gram_name': 0.7014475399546076, 'clust_count_name': 58.580280148669736, 'clust_count_2gram_name': 0.6727102959902191, 'clust_description': 6.888706024354501, 'clust_description_s': 3.7709706372309437, 'clust_2gramm_description': 3.0600582221709085, 'clust_3gramm_description': 0.9056

### собираем части в единый результат

In [ ]:
y_result_load = np.load('y_result'+str(10)+'.npy')
for n in range(20,621,10):
    fn = 'y_result'+str(n)+'.npy'
    y_result_load = np.vstack((y_result_load,np.load(fn))).astype('float16')
    print(fn)
np.save('y_result_final',y_result_load)
y_result_load.shape

### по подобранному порогу отбираем вакансии

In [68]:
%%time
gc.collect()
y_result = np.load('y_result_final.npy')

for i in tqdm(range(0,y_result.shape[1])):

    dd=0.31
    cury = y_result[:,i]
    qty = len(cury[cury>dd])
        
    while ((qty<=0)|(qty>6)):
        if qty<=0:
            if dd>0.05:
                dd -= 0.005
            else:
                dd -= 0.001
        if qty>6:
            dd += 0.005
        qty = len(cury[cury>dd])
    cury[cury>dd] = 1
    cury[cury<=dd] = 0

#формируем список определенных специализаций
d = y_result.T*mlb.classes_

#формируем строки предсказаний в словарь (так быстрее всего получилось)
y_pred = {}
for k in tqdm(range(0,d.shape[0])):
    ddd = d[k]
    y_pred[df_test.iloc[k,0]] = np.array2string(ddd[ddd>0].astype('int'),separator=',')
print(len(y_pred))

#из словаря формируем окончательный предсказанный датасет
df_fin = pd.DataFrame(y_pred,index=[0]).T
#записываем датасет в файл
df_fin.reset_index().rename(columns={'index':'vacancy_id',0:'specializations'}).to_csv('submisson.csv',sep=',',index=False)


1456325
Wall time: 2min 59s
